In [27]:
import numpy as np
import pandas as pd
import gc

train = pd.read_csv('../data/train_2016_v2.csv')
prop = pd.read_csv('../data/properties_2016.csv')
sample = pd.read_csv('../data/sample_submission.csv')

In [28]:
"""メモリ節約のためfloat64 -> 32
"""
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [59]:
df_train = train.merge(prop, on='parcelid')

X = df_train.drop([
    'parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'
], axis=1).fillna(0)

y = df_train['logerror'].astype(np.float32).values

X.shape, y.shape

((90275, 55), (90275,))

In [60]:
from sklearn.model_selection import train_test_split

train_columns = X.columns

for c in X.dtypes[X.dtypes == object].index.values:
    X[c] = (X[c] == True)

del df_train; gc.collect()

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((72220, 55), (18055, 55), (72220,), (18055,))

In [76]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [77]:
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

y_pred = model.predict(x_test.fillna(0))
y_pred

/Users/a13446/.anyenv/envs/pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


array([-0.00951976, -0.0145923 ,  0.65804912, ...,  0.11483336,
        0.11483336,  0.11483336])

In [80]:
sub = pd.read_csv('../data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = y_pred

print('Writing csv ...')
sub.to_csv('linear_reg_starter.csv', index=False, float_format='%.4f') # Thanks to @inversion

Writing csv ...


In [82]:
np.mean(y_pred**2)

0.00079534362915160116

In [84]:
!gzip linear_reg_starter.csv